# Типы данных

---


## (1) Введение

---


**О чём будем говорить**

Каждый раз, когда мы обсуждали хранящиеся в рабочих таблицах данные, мы обязательно указывали их принадлежность к тому или иному типу.

Прежде мы не акцентировали на этом внимание, но в действительности знание этих типов крайне важно для всего рабочего процесса. И этот модуль мы посвятим разговору о типах данных, а также об особенностях взаимодействия с некоторыми из них.

В этом модуле вы:

- получите общие сведения об основных типах данных, которые применяются для хранения информации в базах данных;
- разберётесь в специфике строк и дат;
- узнаете, какие функции и операторы можно использовать для решения задач со строками и датами.


**С чем будем работать**

В этом модуле мы будем работать с уже знакомой вам схемой shipping, хранящей таблицы с данными о компании, которая организует перевозки грузов.

Интересующие нас данные всё так же хранятся в таблицах city, customer, driver и shipment (таблица truck в этом модуле нам не понадобится).

![](data/dst3-u2-md5_1_1.png)


При необходимости вы можете освежить воспоминания о структуре и содержимом датасета, вернувшись к описанию таблиц в модуле SQL-04:

- [локальная ссылка](</home/user/IDE/sf_data_science/synopsis/block_2/SQL-04/02_sql_04_theory.ipynb##_(1)_Знакомство_с_данными>)

- [ссылка GitHub](<https://github.com/Stanislav-DS/sf_data_science/blob/main/synopsis/block_2/SQL-04/02_sql_04_theory.ipynb##(1)_Знакомство_с_данными>)


## (2) Типы данных в PostgreSQL

---


Особенностью хранения данных в БД является их строгая типизация, то есть точное и явное определение типов. Необходимость в типизации обусловлена тем, что компьютер по-разному обрабатывает даты, целые или дробные числа, строки.

Поэтому при создании таблицы БД обязательно указывают типы данных, которые будут сохраняться в каждом из её полей, — от этого зависит, какие значения допустимы в этих полях.

Если поле таблицы определено числовым типом данных, то произвольную строку 'Всем привет!' в него записать уже не получится. Таким образом, типы данных — это один из основных видов ограничений в PostgreSQL.

Подробнее об ограничениях, или constraints, вы можете почитать в [официальной документации PostgreSQL](https://postgrespro.ru/docs/postgresql/9.5/ddl-constraints).


> **SQL** — это язык со строгой типизацией, в котором каждый элемент данных имеет некоторый тип, определяющий его поведение и допустимое использование.

Типы данных в PostgreSQL можно разделить на несколько групп. К основным относятся:

- числовые типы — для хранения чисел (целых и дробных);
- типы даты/времени — для хранения даты, времени, часовых поясов;
- символьные типы — для хранения символов или строк;
- логический тип — для хранения значений типа «истина», «ложь».

Каждая группа (кроме логического типа) объединяет несколько типов, отличающихся по допустимому диапазону хранимых данных. Например, числовые типы данных хранят целые числа, дробные числа, а строковые — подразделяются на типы с фиксированной и переменной длиной. А скажем, целочисленный тип данных _integer_ может хранить значения в диапазоне от -2147483648 до 2147483647.

Конечно, чем больше допустимый диапазон, тем больше требуется места на жёстком диске для хранения каждого поля выбранного типа.

Кроме возможных значений из доступного диапазона, поле независимо от выбранного типа может принимать значение _NULL_ (отсутствующее значение).

Напоминаем, что NULL отличается от нулевого значения или поля, содержащего пробел или пустую строку.


В зависимости от требований к хранимой информации необходимо правильно применять типы данных. На это есть как минимум две причины.

1. Разные типы данных могут занимать разный объём памяти.

   Например, если заранее известно, что поле таблицы будет принимать только небольшие числовые значения от 1 до 10, то можно не задавать для него тип _bigint_ с максимально возможными хранимыми значениями, достаточно будет типа _smallint_ (эти типы не будут предметом нашего обсуждения, но вы может посмотреть их описание в таблице).

2. На преобразование типов данных тратится время.

   В некоторых случаях может потребоваться преобразование одного типа в другой — например, если нужно извлечь число из строки и произвести с ним какие-либо действия. Для подобных преобразований нужно использовать дополнительные функции. Иногда без этого не обойтись, но по возможности лучше хранить данные в полях соответствующего типа.

В этом модуле мы ограничимся разговором о таких типах данных, как даты и строки, а также поговорим о функциях, которые позволяют с ними работать.

Максимально подробную информацию по всем существующим типам данных вы можете посмотреть в [документации](https://postgrespro.ru/docs/postgrespro/9.5/datatype).


## (3) Даты: основные типы.

---


Любой анализ событий во времени подразумевает работу с датой или временем.

Для них в `Postgres` существует несколько типов данных — все они представлены в таблице ниже.

| Имя                             | Размер  | Описание                              | Наименьшее значение | Наибольшее значение | Точность                 |
| ------------------------------- | ------- | ------------------------------------- | ------------------- | ------------------- | ------------------------ |
| timestamp [ without time zone ] | 8 байт  | дата и время (без часового пояса)     | 4713 до н. э.       | 294276 н. э.        | 1 микросекунда / 14 цифр |
| timestamp with time zone        | 8 байт  | дата и время (с часовым поясом)       | 4713 до н. э.       | 294276 н. э.        | 1 микросекунда / 14 цифр |
| date                            | 4 байта | дата (без времени суток)              | 4713 до н. э.       | 5874897 н. э.       | 1 день                   |
| time [ without time zone ]      | 8 байт  | время суток (без даты)                | 0:00:00             | 24:00:00            | 1 микросекунда / 14 цифр |
| time with time zone             | 12 байт | только время суток (с часовым поясом) | 00:00:00+1459       | 24:00:00-1459       | 1 микросекунда / 14 цифр |
| interval                        | 16 байт | временной интервал                    | -178000000 лет      | 178000000 лет       | 1 микросекунда / 14 цифр |


Давайте подробно разберём каждый из этих типов и выясним, в каких случаях и для чего они используются.


**TIMESTAMP**

> **Timestamp** — наиболее распространённый тип данных, так как он содержит и дату, и время, а также используется в любых логах событий, временных рядах и в большинстве системных таблиц.

Согласно [стандарту ISO](https://ru.wikipedia.org/wiki/ISO_8601), значение выглядит как **"2019-07-14 01:35:44.702165+00"**, где перечислены через точку год-месяц-день, время и часовой пояс.

Для получения текущего значения даты и времени в _Postgres_ используются функции `CURRENT_TIMESTAMP` (есть в стандарте _SQL_) и `NOW()` (есть в большинстве баз данных).

Выполните запросы `SELECT NOW()` и `SELECT CURRENT_TIMESTAMP`.

Глядя на вывод, вы заметите, что время отличается на целое число часов от реального времени у вас (если вы не в Лондоне или, скажем, на Канарских островах). В Москве разница составит 3 часа.

Почему так?

Давайте разберёмся.


**TIMESTAMP WITH TIME ZONE**

> **Timestamp with time zone** позволяет хранить сведения о часовом поясе, что может быть удобно при анализе географически распределённых временных данных для единообразия хранения.

Предположим, в вашей компании в базу подтягивается время прихода сотрудников на работу. Вы пришли в 10 утра по Москве, а в Екатеринбурге в это время — полдень. Чтобы ваши коллеги из Екатеринбурга поняли, что вы пришли на работу вовремя, им нужно помнить про разницу в часовых поясах. А теперь представим, что пользователи БД разбросаны по всему миру и всем им необходимо помнить о разнице во времени и учитывать её при сверках с другими регионами.

Согласитесь, неудобно. Поэтому временные метки лучше всего хранить в базе в едином часовом поясе.

Посмотрим, как работает этот тип данных.

Сначала попробуем узнать, в каком часовом поясе выводятся временные данные в настоящий момент. Для этого выполните команду

```sql
SHOW TIMEZONE
```

В результате вы увидите **GMT** — это наиболее частая установка по умолчанию для баз данных. Что он обозначает, можно посмотреть [здесь](https://ru.wikipedia.org/wiki/%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%B5_%D0%B2%D1%80%D0%B5%D0%BC%D1%8F_%D0%BF%D0%BE_%D0%93%D1%80%D0%B8%D0%BD%D0%B2%D0%B8%D1%87%D1%83).

Список часовых поясов можно увидеть в системном справочнике `pg_timezone_names`.

| Имя        | Тип      | Описание                                                                                      |
| ---------- | -------- | --------------------------------------------------------------------------------------------- |
| name       | text     | Название часового пояса                                                                       |
| abbrev     | text     | Сокращение часового пояса                                                                     |
| utc_offset | interval | Смещение от GMT (положительное — к востоку от Гринвича, отрицательное — к западу от Гринвича) |
| is_dst     | boolean  | Принимает значение True, если текущие данные отражают летнее время                            |


Описание этого справочника вы не сможете посмотреть в Metabase, но можете к нему обратиться, написав SQL-запрос.

Посмотрите содержимое этого справочника в Metabase.

А ещё посмотрите ваше время в каком-нибудь часовом поясе, например, в Москве. Для этого выполните в Metabase запрос

```sql
SELECT NOW() AT TIME ZONE 'Europe/Moscow'
```

Указание `AT TIME ZONE` позволяет переводить дату/время без часового пояса в дату/время с часовым поясом и обратно, а также пересчитывать значения времени для различных часовых поясов.


В таблице ниже приведены примеры того, как работает `AT TIME ZONE` для разных типов данных.

| Выражение                                               | Тип результата              | Описание                                                                                                            |
| ------------------------------------------------------- | --------------------------- | ------------------------------------------------------------------------------------------------------------------- |
| `timestamp without time zone AT TIME ZONE` часовой_пояс | timestamp with time zone    | Воспринимает время, заданное без указания часового пояса, как время в указанном часовом поясе.                      |
| `timestamp with time zone AT TIME ZONE` часовой_пояс    | timestamp without time zone | Переводит значение timestamp с часовым поясом в другой часовой пояс, но не сохраняет информацию о нём в результате. |
| `time with time zone AT TIME ZONE` часовой_пояс         | time with time zone         | Переводит время с часовым поясом в другой часовой пояс.                                                             |


**Задание 3.1**

> Давайте узнаем, сколько сейчас времени в другом регионе, например Лос-Анджелесе. Напишите запрос, который выведет текущие время и дату в часовом поясе Лос-Анджелеса (`"America/Los_Angeles"`). Столбец в выдаче — now (время и дата в нужном часовом поясе).


**Решение**

```sql
SELECT NOW() AT TIME ZONE 'America/Los_Angeles'
```


**DATE**

С типом _date_ вы уже знакомы, его реализация предельно проста. Отметим только, что тип _timestamp_ (_with/without time zone_) можно легко перевести в соответствующую дату, используя синтаксис:

```sql
"timestamp_column"::date
```

И наоборот, тип _date_ преобразуется в _timestamp (дата и 00:00:00)_ с помощью:

```sql
"date_column"::timestamp
```

Для получения текущей даты можно использовать:

```sql
select CURRENT_DATE
```

или

```sql
select now()::date
```


**Задание 3.2**

> Предположим, у нас есть дата и время какого-то события и мы хотим посмотреть, к какой дате оно относится для Москвы и для UTC. Используйте следующий подзапрос и выведите дату в `ts` в Московском часовом поясе и в поясе UTC:

```sql
WITH x AS (
    SELECT
        '2018-12-31 21:00:00+00' :: TIMESTAMP WITH TIME ZONE AS ts
)
```

> Столбцы в выдаче: `dt_msk` (дата в московском часовом поясе), `dt_utc` (дата в UTC).
>
> **Примечание:** в данном куске SQL-кода мы обозначаем результат запроса SQL как таблицу с именем x. В этой таблице содержится столбец ts. Воспользуйтесь этой таблицей для решения задачи. Например, чтобы посмотреть всё содержимое таблицы x, достаточно написать следующий запрос:

```sql
WITH x AS (
    SELECT
        '2018-12-31 21:00:00+00' :: TIMESTAMP WITH TIME ZONE AS ts
)
SELECT
    *
FROM
    x
```

Узнать больше об операторе `WITH` можно [здесь](https://sql-academy.org/ru/guide/operator-with)


**Решение**

```sql
with x as (
    select
        '2018-12-31 21:00:00+00' :: timestamp with time zone as ts
)
select
    x.ts at time zone 'Europe/Moscow' :: date as dt_msk,
    x.ts at time zone 'UTC' :: date as dt_utc
from
    x
```


**INTERVAL**

> **Interval** — тип данных, позволяющий хранить разницу между двумя временными метками.

Интервалы хранят данные в трёх отдельных полях — месяцах, днях, секундах. Это сделано из-за того, что количество дней в месяце и часов в дне может быть разным. Пример значения такого типа: **"195 days -10:52:23.563955"**.

Подробнее о том, как работают интервалы, мы узнаем позже, на примере конкретных функций, с помощью которых можно вычислять эти интервалы и работать с ними.


## (4) Функции и операторы для работы с датами.

---


### Функция EXTRACT()


Функция `extract()` получает из значений даты/времени такие поля, как год или час.

Здесь источник — значение типа `timestamp`, `time` или `interval`. Допускается и тип `date`, поскольку он приводится к типу timestamp.

Указанное поле представляет собой идентификатор, по которому из источника выбирается заданное поле. Функция `extract()` возвращает значения типа _double precision_.


**DAY**

Для значений `timestamp` это день месяца (1-31), для значений `interval` — число дней.

```sql
SELECT
    EXTRACT(
        DAY
        FROM
            TIMESTAMP '2001-02-16 20:38:40'
    );
```

**Результат:** 16

```sql
SELECT
    EXTRACT(
        DAY
        FROM
            INTERVAL '40 days 1 minute'
    );
```

**Результат:** 40


**HOUR**

Час (0-23).

```sql
SELECT
    EXTRACT(
        HOUR
        FROM
            TIMESTAMP '2001-02-16 20:38:40'
    );
```

**Результат:** 20


**MONTH**

Номер месяца, считая с января (1) до декабря (12).

```sql
SELECT
    EXTRACT(
        MONTH
        FROM
            TIMESTAMP '2001-02-16 20:38:40'
    );
```

**Результат:** 2001


**YEAR**

Поле года. Учтите, что года 0 не было, и это следует иметь в виду, вычитая из годов нашей эры годы до нашей эры.

```sql
SELECT
    EXTRACT(
        YEAR
        FROM
            TIMESTAMP '2001-02-16 20:38:40'
    );
```

**Результат:** 2001


**ISOYEAR**

Год по недельному календарю _ISO 8601_, в который попадает дата (не применимо к интервалам).

```sql
SELECT
    EXTRACT(
        ISOYEAR
        FROM
            DATE '2006-01-01'
    );
```

**Результат:** 2005

```sql
SELECT
    EXTRACT(
        ISOYEAR
        FROM
            DATE '2006-01-02'
    );
```

**Результат:** 2006

Год по недельному календарю _ISO_ начинается с понедельника недели, в которой оказывается 4 января, так что в начале января или в конце декабря год по _ISO_ может отличаться от года по григорианскому календарю. Подробнее об этом рассказывается в описании поля _week_.


**WEEK**

Номер недели в году по недельному календарю _ISO 8601_. По определению, недели _ISO 8601_ начинаются с понедельника, а первая неделя года включает 4 января этого года. Другими словами, первый четверг года всегда оказывается в первой неделе этого года.

В системе нумерации недель _ISO_ первые числа января могут относиться к 52-й или 53-й неделе предыдущего года, а последние числа декабря — к первой неделе следующего года.

**Например**, _2005-01-01_ относится к 53-й неделе 2004 г., а _2006-01-01_ — к 52-й неделе 2005 г., тогда как _2012-12-31_ включается в первую неделю 2013 г.

Поэтому для получения согласованных результатов рекомендуется использовать поле `isoyear` в паре с `week`.

```sql
SELECT
    EXTRACT(
        WEEK
        FROM
            TIMESTAMP '2001-02-16 20:38:40'
    );
```

**Результат:** 7


**CENTURY**

```sql
SELECT
    EXTRACT(
        CENTURY
        FROM
            TIMESTAMP '2001-02-16 20:38:40'
    );
```

**Результат:** 21

Первый век начался 0001-01-01 00:00:00, хотя люди в то время так и не считали. Это определение распространяется на все страны с григорианским календарём.

Века с номером 0 не было; считается, что 1 наступил после -1.

Если такое положение вещей вас не устраивает, направляйте жалобы по адресу: Ватикан, Собор Святого Петра, Папе Римскому, лично в руки :)


**DECADE**

Десятилетие.

```sql
SELECT
    EXTRACT(
        DECADE
        FROM
            TIMESTAMP '2001-02-16 20:38:40'
    );
```

**Результат:** 200


**EPOCH**

Для значений `timestamp with time zone` это число секунд с _1970-01-01 00:00:00 UTC_ (может быть отрицательным); для значений `date` и `timestamp` это число секунд с _1970-01-01 00:00:00_ по местному времени, а для `interval` — общая длительность интервала в секундах.

```sql
SELECT
    EXTRACT(
        EPOCH
        FROM
            TIMESTAMP WITH TIME ZONE '2001-02-16 20:38:40.12-08'
    );
```

**Результат:** 982384720.12

```sql
SELECT
    EXTRACT(
        EPOCH
        FROM
            INTERVAL '5 days 3 hours'
    );
```

**Результат:** 442800

Преобразовать время эпохи обратно, в значение дата/время, с помощью `to_timestamp` можно так:

```sql
SELECT
    to_timestamp(982384720.12);
```

**Результат:** 2001-02-17 04:38:40.12+00


**DOW**

День недели, считая с воскресенья (0) до субботы (6).

```sql
SELECT
    EXTRACT(
        DOW
        FROM
            TIMESTAMP '2001-02-16 20:38:40'
    );
```

**Результат:** 5

Заметьте, что в `extract()` дни недели нумеруются не так, как в функции `to_char(..., 'D')`.


**DOY**

День года (1-365/366).

```SQL
SELECT
    EXTRACT(
        DOY
        FROM
            TIMESTAMP '2001-02-16 20:38:40'
    );
```

**Результат:** 47


**ISODOW**

День недели, считая с понедельника (1) до воскресенья (7).

```sql
SELECT
    EXTRACT(
        ISODOW
        FROM
            TIMESTAMP '2001-02-18 20:38:40'
    );
```

**Результат:** 7

Результат отличается от `dow` только для воскресенья. Такая нумерация соответствует _ISO 8601_.


### Задание 4.1


> Давайте посчитаем помесячную статистику по доставкам, используя функцию `extract`. Напишите запрос, который выведет год, месяц и количество доставок. Отсортируйте по году и по месяцу в порядке возрастания. Столбцы в выдаче: `year_n` (номер года), `month_n` (номер месяца), `qty` (количество доставок).


**Решение**

```sql
select
    extract(
        year
        from
            ship_date
    ) as year_n,
    extract(
        month
        from
            ship_date
    ) as month_n,
    count(ship_id) as qty
from
    sql.shipment
group by
    extract(
        year
        from
            ship_date
    ),
    extract(
        month
        from
            ship_date
    )
order by
    year_n,
    month_n
```


### Функция TO_CHAR()


Функция `to_char()` нужна для форматирования даты времени и интервалов в нужный текст.

Например, вы хотите вывести год, месяц и день со специфическим разделителем или получить текстовое наименование месяца или дня недели. По результату работы она очень близка к `extract()`, но больше нацелена именно на форматирование. Ниже вы видите таблицу с примерами вызовов:

| Функция                       | Тип результата | Описание                     | Пример                                       |
| ----------------------------- | -------------- | ---------------------------- | -------------------------------------------- |
| to_char(timestamp[date],text) | text           | преобразует время в текст    | to_char(current_timestamp, 'HH12:MI:SS')     |
| to_char(interval, text)       | text           | преобразует интервал в текст | to_char(interval '15h 2m 12s', 'HH24:MI:SS') |


Первый параметр для функции — дата, время или интервал, а второй — текстовая маска получаемого результата. Перечень значений, которые можно использовать для маски, представлен в таблице ниже.

| Код форматирования | Описание                                                                               |
| ------------------ | -------------------------------------------------------------------------------------- |
| HH24               | час (00-23)                                                                            |
| MI                 | минута (00-59)                                                                         |
| SS                 | секунда (00-59)                                                                        |
| MS                 | миллисекунда (000-999)                                                                 |
| SSSS               | число секунд с начала суток (0-86399)                                                  |
| YYYY               | год (4 или более цифр)                                                                 |
| YY                 | последние две цифры года                                                               |
| month              | полное название месяца в нижнем регистре (дополненное пробелами до девяти символов)    |
| MM                 | номер месяца (01-12)                                                                   |
| day                | полное название дня недели в нижнем регистре, дополненное пробелами до девяти символов |
| DDD                | номер дня в году (001-366)                                                             |
| DD                 | день месяца (01-31)                                                                    |
| ID                 | номер дня недели по ISO 8601, считая с понедельника (1) до воскресенья (7)             |
| WW                 | номер недели в году (1-53); первая неделя начинается в первый день года                |


####

Полный перечень вы можете посмотреть в [подсказке по кодам форматирования](https://postgrespro.ru/docs/postgrespro/9.5/functions-formatting) или таблице (чтобы увидеть её кликните `>`).


| Код форматирования        | Описание                                                                                                         |
| ------------------------- | ---------------------------------------------------------------------------------------------------------------- |
| HH                        | час (01-12)                                                                                                      |
| HH12                      | час (01-12)                                                                                                      |
| HH24                      | час (00-23)                                                                                                      |
| MI                        | минута (00-59)                                                                                                   |
| SS                        | секунда (00-59)                                                                                                  |
| MS                        | миллисекунда (000-999)                                                                                           |
| US                        | микросекунда (000000-999999)                                                                                     |
| SSSS                      | число секунд с начала суток (0-86399)                                                                            |
| AM, am, PM или pm         | обозначение времени до/после полудня (без точек)                                                                 |
| A.M., a.m., P.M. или p.m. | обозначение времени до/после полудня (с точками)                                                                 |
| Y,YYY                     | год (четыре или более цифр) с разделителем                                                                       |
| YYYY                      | год (четыре или более цифр)                                                                                      |
| YYY                       | последние три цифры года                                                                                         |
| YY                        | последние две цифры года                                                                                         |
| Y                         | последняя цифра года                                                                                             |
| IYYY                      | недельный год по ISO 8601 (четыре или более цифр)                                                                |
| IYY                       | последние три цифры недельного года по ISO 8601                                                                  |
| IY                        | последние две цифры недельного года по ISO 8601                                                                  |
| I                         | последняя цифра недельного года по ISO 8601                                                                      |
| BC, bc, AD или ad         | обозначение эры (без точек)                                                                                      |
| B.C., b.c., A.D. или a.d. | обозначение эры (с точками)                                                                                      |
| MONTH                     | полное название месяца в верхнем регистре (дополненное пробелами до девяти символов)                             |
| Month                     | полное название месяца с большой буквы (дополненное пробелами до девяти символов)                                |
| month                     | полное название месяца в нижнем регистре (дополненное пробелами до девяти символов)                              |
| MON                       | сокращённое название месяца в верхнем регистре (три буквы в английском; в других языках длина может меняться)    |
| Mon                       | сокращённое название месяца с большой буквы (три буквы в английском; в других языках длина может меняться)       |
| mon                       | сокращённое название месяца в нижнем регистре (три буквы в английском; в других языках длина может меняться)     |
| MM                        | номер месяца (01-12)                                                                                             |
| DAY                       | полное название дня недели в верхнем регистре (дополненное пробелами до девяти символов)                         |
| Day                       | полное название дня недели с большой буквы (дополненное пробелами до девяти символов)                            |
| day                       | полное название дня недели в нижнем регистре (дополненное пробелами до девяти символов)                          |
| DY                        | сокращённое название дня недели в верхнем регистре (три буквы в английском; в других языках может меняться)      |
| Dy                        | сокращённое название дня недели с большой буквы (три буквы в английском; в других языках длина может меняться)   |
| dy                        | сокращённое название дня недели в нижнем регистре (три буквы в английском; в других языках длина может меняться) |
| DDD                       | номер дня в году (001-366)                                                                                       |
| IDDD                      | номер дня в году по ISO 8601 (001-371; 1 день — понедельник первой недели по ISO)                                |
| DD                        | день месяца (01-31)                                                                                              |
| D                         | номер дня недели, считая с воскресенья (1) до субботы (7)                                                        |
| ID                        | номер дня недели по ISO 8601, считая с понедельника (1) до воскресенья (7)                                       |
| W                         | неделя месяца (1-5) (первая неделя начинается в первое число месяца)                                             |
| WW                        | номер недели в году (1-53) (первая неделя начинается в первый день года)                                         |
| IW                        | номер недели в году по ISO 8601 (01-53; первый четверг года относится к неделе 1)                                |
| CC                        | век (две цифры; двадцать первый век начался 2001-01-01)                                                          |
| J                         | день по юлианскому календарю (номер дня с 24 ноября 4714 г. до н. э.)                                            |
| Q                         | квартал (игнорируется функциями to_date и to_timestamp)                                                          |
| RM                        | номер месяца римскими цифрами в верхнем регистре (I-XII; I=январь)                                               |
| rm                        | номер месяца римскими цифрами в нижнем регистре (i-xii; i=январь)                                                |
| TZ                        | сокращённое название часового пояса в верхнем регистре (поддерживается только в to_char)                         |
| tz                        | сокращённое название часового пояса в нижнем регистре (поддерживается только в to_char)                          |
| OF                        | смещение часового пояса от UTC (поддерживается только в to_char)                                                 |


####


> Любой текст, который относится к кодам форматирования, будет сохранён как есть. Чтобы оставлять какие-то подсказки и использовать обычные буквы, которые не встречаются в таблице выше, необходимо заключить текст, который нужно оставить без изменений, **в двойные кавычки.**


Предположим, мы хотим вывести сегодняшнюю дату в формате _"Hello! Today is #название дня недели год.название месяца.день#"_ текстом. Для этого нужно выполнить следующий код:

```sql
select
    to_char(now(), '"Hello! Today is" DAY yyyy-Mon-dd')
```

Также поэкспериментируйте с разными видами форматирования и произвольными текстами.


### Задание 4.2


Давайте выведем текст текущего времени для сервиса точного времени. Напишите запрос, который выводит текст _"Точное время x часов y минут z секунд"_ (текст в кавычки заключать не нужно), где x, y, z — часы, минуты и секунды соответственно, при условии, что сообщение нужно вывести для московского часового пояса. Время введите в 24-часовом формате. Столбцы в выдаче: `msg` (сообщение).


**Решение**

```sql
select
    to_char(
        now() at time zone 'Europe/Moscow',
        '"Точное время" HH24 "часов" MI "минут" SS "секунд"'
    ) as msg;
```


### Функция DATE_TRUNC()


Функция `date_trunc()` позволяет отсечь заданное время, дату или дату со временем до нужной точности.

Формат вызова:
`date_trunc('поле', значение)`

Например, если мы хотим округлить текущее время-дату до минут, то можно вызвать

```sql
select
    date_trunc('minute', now())
```


Для получения разной степени точности вместо minute можно использовать следующие параметры:

- microseconds;
- milliseconds;
- second;
- minute;
- hour;
- day;
- week;
- month;
- quarter;
- year;
- decade;
- century;
- millennium.

Из их названий легко понять, какая временная единица подразумевается.


### Задание 4.3


Давайте подготовим данные для квартальной отчётности компании. Напишите запрос, который выведет дату доставки, округлённую до квартала, и общую массу доставок. Отсортируйте по кварталу в порядке возрастания. Столбцы в выдаче: `q` (начало квартала, тип `date`), `total_weight` (сумма масс доставок за квартал).


**Решение**

```sql
select
    date_trunc('quarter', shipment.ship_date) :: date as q,
    sum(shipment.weight) as total_weight
from
    sql.shipment as shipment
group by
    date_trunc('quarter', shipment.ship_date)
order by
    q
```


**Решение 2 (более наглядное представление)**

```sql
select
    to_char(
        date_trunc('quarter', shipment.ship_date) :: date,
        'Q"-й кв-л" YYYY "г."'
    ) as q,
    sum(shipment.weight) as total_weight
from
    sql.shipment as shipment
group by
    date_trunc('quarter', shipment.ship_date) :: date
order by
    date_trunc('quarter', shipment.ship_date) :: date
```


### Математические операторы


К любой дате можно прибавить (и вычесть из неё) целое число X и получить другую дату, которая больше (меньше) изначальной.

Пример:

```sql
select
    '2019-01-01' :: date + 10
```

**Результат:** '2019-01-11'

Это — дата на 10 дней позже 2019-01-01.

При добавлении (или вычитании) целого числа к дате Postgres учитывает переходы между месяцами и годами и даёт верный ответ, соответствующий календарю. Учитываются даже високосные годы.

Пример:

```sql
select
    '2019-01-01' :: date + 500
```

**Результат:** '2020-05-15'

Как видим, сменились и год, и месяц, и день.


Аналогично можно вычесть из одной даты другую и получить расстояние в днях между этими датами. При такой операции тоже будет честная разница по календарю.

Пример:

```sql
select
    '2019-02-10' :: date - '2017-03-01' :: date
```

**Результат:** 711


### Задание 4.4


Давайте оценим, в каком интервале совершались доставки в разных городах. Напишите запрос, который выведет разницу между последним и первым днём доставки по каждому городу. Отсортируйте по первому и второму столбцам. Столбцы в выдаче: `city_name` (название города) и `days_active` (время от первой до последней доставки в днях).


**Решение (переусложнённое)**

```sql
with first_sh as (
    select
        city_id,
        min(ship_date) as date
    from
        sql.shipment
    group by
        city_id
),
last_sh as (
    select
        city_id,
        max(ship_date) as date
    from
        sql.shipment
    group by
        city_id
)
select
    city.city_name,
    last_sh.date - first_sh.date as days_active
from
    sql.city as city
    join first_sh on city.city_id = first_sh.city_id
    join last_sh on city.city_id = last_sh.city_id
group by
    city.city_id,
    last_sh.date,
    first_sh.date
order by
    1,
    2
```


**Эталонное решение**

```sql
SELECT
    c.city_name,
    max(s.ship_date) - min(s.ship_date) as days_active
FROM
    shipping.shipment s
    JOIN shipping.city c ON c.city_id = s.city_id
GROUP BY
    1
ORDER BY
    1
```


## (5) Строковые данные: основные типы

---


В Postgres есть три основных типа данных для работы со строками: **character, character varying и text**.

Кратко представим каждый из них.

1. `character`

   Cтрока фиксированной длины, дополненная пробелами.

   Длина строки такого типа всегда одинакова и задаётся в скобках.

   Например, в столбце `character(5)` всегда будет пять символов: строку большей длины туда вставить не получится, а строка меньшей длины будет дополняться ведущими пробелами. Слово "SQL" в таком столбце будет выглядеть как " SQL".

   Основной паттерн использования такого типа — универсальные справочники буквенных кодов, например код страны в стандарте _ISO_ (_RU, US, UK_ и т. д.).

2. `character varying`

   Строка ограниченной переменной длины.

   Например, в столбце типа `character varying(5)` нельзя будет хранить строку большей длины, но могут быть любые строки с меньшей длиной.

   Этот тип данных повсеместно используется для хранения данных, поскольку позволяет ограничить ввод, сохраняя при этом возможность иметь строки произвольной длины.

3. `text`

   Cтрока неограниченной длины.

   Самый удобный тип для пользователя, но самый тяжеловесный для администратора баз данных, так как в строку можно записать любой текст.

   Для удобства все текстовые поля в нашем датасете с доставками представлены типом `text`.


## (6) Функции и операторы для работы со строками

---


### Операторы


**Соединение строк**

Для начала познакомимся с оператором конкатенации строк — `||` (две вертикальные черты). Он позволяет объединять две и более строки.

Конструкции с оператором соединения строк записываются следующим образом:

```
строка1 || строка2 || ... || строкаN
```

> Важно! Результатом соединения любых типов строковых данных будет тип `text`.



Напишем запрос, который позволит подготовить простые select-запросы для всех таблиц из схемы.

```sql
select
    'select * from ' || t.table_schema || '.' || t.table_name || ';' query
from
    information_schema.tables t
where
    table_schema = 'shipping'
```

В результате должно получиться пять SQL-запросов, по одному к каждой таблице из схемы shipping. 

Как мы видим, соединять можно и рукописный текст, и значения столбцов в любом произвольном порядке.

> Важно! Если вы соединяете любую строку и *NULL*, то результатом будет *NULL*. Поэтому, если вы формируете какой-то текст на основе поля, в котором присутствует *NULL*, используйте оператор [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull).

**Задание 6.1**

> Составим текстовый шаблон сообщения о доставке по конкретному водителю для наших клиентов. Напишите SQL-запрос, который выведет следующее сообщение для каждого водителя по форме:
>
> `Ваш заказ доставит водитель #Имя Фамилия#. Его контактный номер: #Номер#`
>
> Где `#Имя Фамилия#` и `#Номер#` взяты из справочника водителей. Если номер не указан, то выведите прочерк (`-`). Для номеров рекомендуем использовать `COALESCE`. Пример из таблицы для наглядности:
> 
> `Ваш заказ доставит водитель Adel Al-Alawi. Его контактный номер: (901) 947-4433`
>
> Столбец к выдаче — msg (текст сообщения).

**Решение**

```sql
select
    'Ваш заказ доставит водитель ' || d.first_name || ' ' || d.last_name || '. Его контактный номер: ' || COALESCE(d.phone, '-') as msg
from
    shipping.driver as d
```

### Функции

#### `upper(your_text)` и `lower(your_text)`

Функции `upper(your_text)` и `lower(your_text)` переводят каждый символ вашего текста в верхний и нижний регистр соответственно.

Пример:

```sql
select
    upper('Abc') s1,
    lower('xYz') s2
```

Чаще всего эти функции используются для унификации и стандартизации, особенно они актуальны для данных, введённых вручную.

Например, названия города в анкете можно написать разными способами, но символьный состав останется одним и тем же (Москва, москва, МОСКВА).

Результат функций `upper()` и `lower()` — тоже строковый, а значит, к нему можно применять все функции, применимые к этому типу данных.

#### **Задание 6.2**

> Cоставим справочник названий клиентов, у которых более десяти доставок. Данные сохраним в нижнем регистре, чтобы передавать их в другие системы (например, для обзвона), которые не чувствительны к регистру. Напишите запрос, который выводит все id названий клиентов, у которых более десяти доставок, в нижнем регистре. Отсортируйте результат по `cust_id` в порядке возрастания. Столбцы в выдаче: `cust_id` (`id` клиента) и `cust_name` (название клиента в нижнем регистре).

**Решение**

```sql
select
    c.cust_id,
    lower(c.cust_name) as cust_name
from
    shipping.customer as c
    join shipping.shipment as s on c.cust_id = s.cust_id
group by
    c.cust_id
having
    count(s.cust_id) > 10
order by
    c.cust_id
```

**Эталонное решение**

```sql
SELECT
    c.cust_id,
    lower(cust_name) cust_name
FROM
    shipping.customer c
    JOIN shipping.shipment s ON c.cust_id = s.cust_id
GROUP BY
    1,
    2
HAVING
    COUNT(distinct s.ship_id) > 10
ORDER BY
    1
```

#### replace()

Запись строится следующим образом:

```sql
replace(string text, from text, to text)
```

Эта запись означает, что в исходной строке `string` мы заменяем все вхождения строки `from` на строку `to`.

Разберём на примере.

```sql
select replace('малако','а','о')
```

Результат выполнения такого запроса будет молоко, т. е. все буквы «а» в строке «малако» были заменены на «о».

С таким же успехом можно заменять строку, состоящую из нескольких символов.

Сделаем из слова «машина» слово «матрас».

```sql
select replace('машина','шина','трас')
```

В строке `'машина'` мы заменили строку `'шина'` на строку `'трас'`.

Результат функции `replace()` — строка, а значит, к ней тоже можно применять все известные нам функции работы со строками.

Если вы хотите удалить из строки какие-то символы, то третьим параметром (to) передайте пустую строку `''`(одинарные кавычки без символа внутри).

Например, сделаем из строки "Hello, world!" строку "Hello!".

```sql
select replace('Hello, world!',', world','')
```


#### Задание 6.3

> Составим справочник utm-меток, для того чтобы передавать город и штат прямо в адресной строке. (Если вы не знаете, что такое utm-метка, почитайте статью на Вики. К программе курса это не относится, но знать полезно.) Напишите SQL-запрос, который выведет список сочетаний из справочника следующего вида: название_штата__название_города, где названия штата и города взяты из справочника городов и переведены в нижний регистр. Столбец к выдаче — utm (форматированный штат-город). Отсортируйте полученный справочник по алфавиту. Обратите внимание! Все пробелы в названиях городов и штатов замените символом '_' (одно нижнее подчёркивание), а для разделения названий города и штата используйте '__' (два последовательных нижних подчёркивания). Пример из таблицы для наглядности: new_jersey__union_city